In [ ]:
!pip install spacy transformers nltk rdflib
!python -m spacy download en_core_web_sm

import spacy
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import nltk
from nltk.corpus import wordnet as wn
import json
from rdflib import Graph, Namespace, RDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.1/565.1 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 52.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
# download them running the first time
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
# !huggingface-cli login
from huggingface_hub import login
login(token="this is the place of your token")


In [ ]:
# model loading
# not good: model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
# not good: model_name = "RAG+HuggingFaceH4/zephyr-7b-alpha"
import torch

nlp = spacy.load("en_core_web_sm")
model_name = "mistralai/Mistral-7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype=torch.float16) #torch_dtype="auto"

# generator = pipeline("text-generation", model=model, tokenizer=tokenizer)
llm = pipeline("text-generation", model=model, tokenizer=tokenizer, return_full_text=False)



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
# from json_repair import repair_json
import torch
import json, ast, re, time
from typing import List

from difflib import SequenceMatcher

# max_new_tokens=256,
llm = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=1024, return_full_text=False)

def extract_verbs(paragraph: str) -> list:
    doc = nlp(paragraph)
    verbs = {token.lemma_ for token in doc if token.pos_ in ("VERB", "AUX") }  # using lemma_ to get the original prototype
    return list(verbs)

def extract_phrases(paragraph: str) -> list:
    doc = nlp(paragraph)
    return list({chunk.text for chunk in doc.noun_chunks})

def clean_phrases(phrases: list) -> list:
    """
    remove duplications and keep the original order
    """
    seen = set()
    cleaned = []
    for p in phrases:
        if p not in seen:
            seen.add(p)
            cleaned.append(p)
    return cleaned

def parse_raw_json(raw: str, max_retries: int = 3) -> dict:
    """
    Extract the first effective json
    Exception handlings..
    """
    def clean_raw(text: str) -> str:
        text = text.replace("…", "").replace("...", "")
        text = re.sub(r'[^\x00-\x7F]+', ' ', text)  # remove non ASCII characters
        return text.strip()

    def is_valid_triples(triples) -> bool:
        if not isinstance(triples, list):
            return False
        for t in triples:
            if not isinstance(t, dict):
                return False
            if not all(k in t for k in ("subject", "predicate", "object")):
                return False
            if any(v.strip() in ["...", "…", ""] for v in t.values()):
                return False
        return True

    raw = clean_raw(raw)

    for attempt in range(max_retries):
        candidates = re.findall(r'\{.*?\}', raw, re.S)
        for c in candidates:
            try:
                data = json.loads(c)
                if isinstance(data, dict) and "triples" in data and is_valid_triples(data["triples"]):
                    return data
            except json.JSONDecodeError:
                continue

        # tail cutting situation handling..
        start, end = raw.find("{"), raw.rfind("}") + 1
        if start != -1 and end > start:
            try:
                partial = raw[start:end]
                data = json.loads(partial)
                if isinstance(data, dict) and "triples" in data and is_valid_triples(data["triples"]):
                    return data
            except json.JSONDecodeError:
                pass

        raw = raw.rstrip(" ,;")  # abnormal signs

    return {"triples": []}

def dynamic_max_tokens(phrases: list, triple_per_phrase: int = 2,
                       min_tokens: int = 256, max_tokens: int = 1024) -> int:
    """
    According to the number of phrases, decide max_new_tokens
    """
    estimated_triples = max(5, len(phrases) * triple_per_phrase)
    estimated_tokens = estimated_triples * 40  # every triple ~40 tokens
    return max(min_tokens, min(estimated_tokens, max_tokens))

def extract_triplets(phrases: list, verbs: list, paragraph: str, max_retries: int = 3) -> list:

    """
    Extraction and auto retry
    """
    max_tokens = dynamic_max_tokens(phrases)

    # prompt = f"""
    #     Given this paragraph:
    #     {paragraph};
    #     Use ONLY these Candidate Phrases, generate RDF triples for them: {phrases};
    #     Your output should be ONLY valid JSON in this format:
    #     {{
    #       "triples": [
    #         {{ "subject": "...", "predicate": "...", "object": "..." }}
    #       ]
    #     }}
    # """
    prompt = f"""
      You are an RDF triple extractor.

      <<<SENTENCE>>>
      {paragraph}
      <<<END OF SENTENCE>>>

      <<<CANDIDATE_PHRASES>>>
      {json.dumps(phrases, ensure_ascii=False)}
      <<<END OF PHRASES>>>

      <<<CANDIDATE_VERBS>>>
      {json.dumps(verbs, ensure_ascii=False)}
      <<<END OF VERBS>>>

      Instructions:
      - Include All of the candidate phrases as either subjects or objects,
      - Predicates are from CANDIDATE_VERBS,
      - The triple's logic should adhere to the SENTENCE.
      - Make sure to cover as much relations that can be learned from the SENTENCE as possible.
      - Output must be valid JSON (no extra text), following this format:
      {{
        "triples": [
          {{ "subject": "...", "predicate": "...", "object": "..." }},
          {{...}}
        ]
      }}
      """

    print("=== PROMPT ===")
    print(prompt)
    print("======================")

    if not verbs:
        return []

    for attempt in range(max_retries):
        raw = llm(prompt, temperature=0.2, max_new_tokens=max_tokens)[0]['generated_text']
        print(f"=== RAW LLM OUTPUT (Attempt {attempt + 1}) ===\n{raw}\n======================")

        parsed = parse_raw_json(raw)
        if parsed["triples"]:  # checking valid triples
            return parsed["triples"]

        print(f"Attempt {attempt + 1}: No valid JSON found, retrying...")
        time.sleep(1)  # wait for 1 sec to retry

    print("Failed to get valid JSON after retries.")
    return []

def normalize_term(term: str) -> str:
    """
    normalize terms
    - stay the original form for PROPNs or Acronyms
    - Using spaCy's lemma for others
    - accept the result only the similarity number is good enough
    """
    doc = nlp(term)
    out = []
    for tok in doc:
        if tok.pos_ == "PROPN" or tok.is_upper:
            out.append(tok.text)
        else:
            out.append(tok.lemma_)
    candidate = re.sub(r"\s+", " ", " ".join(out)).strip()

    # similarity（avoid the change like: software engineer→programmer ）
    sim = SequenceMatcher(None, term.lower(), candidate.lower()).ratio()
    return candidate if sim >= 0.6 else term   # it can be changed such as 0.5~0.7

def align_concepts(triples: list) -> tuple:
    nodes = set()
    aligned = []
    for t in triples:
        s, p, o = t["subject"], t["predicate"], t["object"]
        # ls, lo = to_lemma(s), to_lemma(o)
        ls, lo = normalize_term(s), normalize_term(o)
        nodes.update([ls, lo])
        aligned.append({
            "subject": ls,
            "predicate": p,
            "object": lo
        })
    return list(nodes), aligned


def split_into_sentences(text: str) -> list[str]:
    if not text or not text.strip():
        return []

    # using punctuations and space to split
    rough = re.split(r'(?<=[.!?;])\s+', text.strip())

    # delete blank fragments
    rough = [s.strip() for s in rough if s and s.strip()]

    # remove abnormals like 1.
    sentences = []
    i = 0
    while i < len(rough):
        cur = rough[i]
        if re.fullmatch(r'\d+\.', cur) and i + 1 < len(rough):
            merged = f"{cur} {rough[i+1]}"
            sentences.append(merged)
            i += 2
        else:
            sentences.append(cur)
            i += 1

    return sentences

def build_ontology(paragraphs: list) -> dict:
    all_triples = []
    orphan_nodes = set()

    for para in paragraphs:
        # sentence by sentence
        sentences = split_into_sentences(para)
        for sent in sentences:
            if not sent.strip():
                continue
            phrases = extract_phrases(sent)
            verbs = extract_verbs(sent)
            phrases = clean_phrases(phrases)
            verbs = clean_phrases(verbs)
            triples = extract_triplets(phrases, verbs, sent)  # using the sentence itself as context
            if triples:
              all_triples.extend(triples)
            else:
              orphan_nodes.update([sent])

    nodes, edges = align_concepts(all_triples)

    node_set = set(nodes) if isinstance(nodes, list) else set()
    for n in orphan_nodes:
        if n not in node_set:
            node_set.add(n)
            nodes.append(n)

    return {"nodes": nodes, "edges": edges}

# examples
if __name__ == "__main__":
    # paragraphs = [
    #     "Software engineering is a field that applies engineering principles to the design, development, and maintenance of software.",
    #     "Ontologies are used to formally represent knowledge within a domain, enabling reasoning and semantic search."
    # ]
    paragraphs = [
        "Software Engineering Code of Ethics and Professional Practice (Short Version)",
        "PREAMBLE",
        "The short version of the code summarizes aspirations at a high level of the abstraction; the clauses that are included in the full version give examples and details of how these aspirations change the way we act as software engineering professionals. Without the aspirations, the details can become legalistic and tedious; without the details, the aspirations can become high sounding but empty; together, the aspirations and the details form a cohesive code.",
        "Software engineers shall commit themselves to making the analysis, specification, design, development, testing and maintenance of software a beneficial and respected profession. In accordance with their commitment to the health, safety and welfare of the public, software engineers shall adhere to the following Eight Principles:",
        "1. PUBLIC – Software engineers shall act consistently with the public interest.",
        "2. CLIENT AND EMPLOYER – Software engineers shall act in a manner that is in the best interests of their client and employer consistent with the public interest.",
        "3. PRODUCT – Software engineers shall ensure that their products and related modifications meet the highest professional standards possible.",
        "4. JUDGMENT – Software engineers shall maintain integrity and independence in their professional judgment.",
        "5. MANAGEMENT – Software engineering managers and leaders shall subscribe to and promote an ethical approach to the management of software development and maintenance.",
        "6. PROFESSION – Software engineers shall advance the integrity and reputation of the profession consistent with the public interest.",
        "7. COLLEAGUES – Software engineers shall be fair to and supportive of their colleagues.",
        "8. SELF – Software engineers shall participate in lifelong learning regarding the practice of their profession and shall promote an ethical approach to the practice of the profession."
    ]
    final_ontology = build_ontology(paragraphs)
    print(json.dumps(final_ontology, indent=2))

    # Success sound
    from IPython.display import Javascript
    def beep():
        display(Javascript('new Audio("https://actions.google.com/sounds/v1/alarms/beep_short.ogg").play()'))
    beep()


<>:259: SyntaxWarning: invalid escape sequence '\d'
<>:259: SyntaxWarning: invalid escape sequence '\d'
/tmp/ipython-input-1983433658.py:259: SyntaxWarning: invalid escape sequence '\d'
  - 避免把列表编号 '1.' 拆成独立句子：若出现单独的 '^\d+\.$' 片段，会与后一个片段合并
Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


=== PROMPT ===

      You are an RDF triple extractor.

      <<<SENTENCE>>>
      Software Engineering Code of Ethics and Professional Practice (Short Version)
      <<<END OF SENTENCE>>>

      <<<CANDIDATE_PHRASES>>>
      ["Software Engineering Code", "Professional Practice", "Ethics", "Short Version"]
      <<<END OF PHRASES>>>

      <<<CANDIDATE_VERBS>>>
      []
      <<<END OF VERBS>>>

      Instructions:
      - Include All of the candidate phrases as either subjects or objects,
      - Predicates are from CANDIDATE_VERBS,
      - The triple's logic should adhere to the SENTENCE.
      - Make sure to cover as much relations that can be learned from the SENTENCE as possible.
      - Output must be valid JSON (no extra text), following this format:
      {
        "triples": [
          { "subject": "...", "predicate": "...", "object": "..." },
          {...}
        ]
      }
      
=== PROMPT ===

      You are an RDF triple extractor.

      <<<SENTENCE>>>
      PREAMBLE
 

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


=== RAW LLM OUTPUT (Attempt 1) ===

      Example output:
      {
        "triples": [
          { "subject": "aspirations", "predicate": "summarize", "object": "the short version" },
          { "subject": "aspirations", "predicate": "summarize", "object": "a high level" },
          { "subject": "aspirations", "predicate": "summarize", "object": "the code" }
        ]
      }
=== PROMPT ===

      You are an RDF triple extractor.

      <<<SENTENCE>>>
      the clauses that are included in the full version give examples and details of how these aspirations change the way we act as software engineering professionals.
      <<<END OF SENTENCE>>>

      <<<CANDIDATE_PHRASES>>>
      ["the full version", "examples", "we", "software engineering professionals", "the clauses", "these aspirations", "that", "details", "the way"]
      <<<END OF PHRASES>>>

      <<<CANDIDATE_VERBS>>>
      ["act", "be", "give", "change", "include"]
      <<<END OF VERBS>>>

      Instructions:
      - Include

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


=== RAW LLM OUTPUT (Attempt 1) ===

      Example:
      {
        "triples": [
          { "subject": "Software engineers", "predicate": "advance", "object": "the integrity and reputation of the profession consistent with the public interest" },
          { "subject": "the integrity", "predicate": "is", "object": "advanced" },
          { "subject": "the profession", "predicate": "is", "object": "advanced" },
          { "subject": "PROFESSION", "predicate": "is", "object": "advanced" },
          { "subject": "reputation", "predicate": "is", "object": "advanced" },
          { "subject": "the public interest", "predicate": "is", "object": "considered" }
        ]
      }
=== PROMPT ===

      You are an RDF triple extractor.

      <<<SENTENCE>>>
      7. COLLEAGUES – Software engineers shall be fair to and supportive of their colleagues.
      <<<END OF SENTENCE>>>

      <<<CANDIDATE_PHRASES>>>
      ["Software engineers", "their colleagues", "7. COLLEAGUES"]
      <<<END OF PHRASE

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


=== RAW LLM OUTPUT (Attempt 1) ===

      Example:
      {
        "triples": [
          { "subject": "Software engineers", "predicate": "be", "object": "fair" },
          { "subject": "Software engineers", "predicate": "be", "object": "supportive" },
          { "subject": "their colleagues", "predicate": "be", "object": "fair" },
          { "subject": "their colleagues", "predicate": "be", "object": "supportive" }
        ]
      }

      Note:
      - The output must be valid JSON, no extra text.
      - The output must adhere to the SENTENCE.
      - The output must cover as much relations that can be learned from the SENTENCE as possible.
=== PROMPT ===

      You are an RDF triple extractor.

      <<<SENTENCE>>>
      8. SELF – Software engineers shall participate in lifelong learning regarding the practice of their profession and shall promote an ethical approach to the practice of the profession.
      <<<END OF SENTENCE>>>

      <<<CANDIDATE_PHRASES>>>
      ["Software en

<IPython.core.display.Javascript object>